In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn import preprocessing

sns.set()

2023-05-01 10:08:42.521970: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 10:08:42.567950: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 10:08:42.568655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 10:08:43.258775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
raw_data = np.loadtxt('./Audiobooks_data.csv',delimiter=',')
unscaled_inputs = raw_data[:,1:-1]
targets_all = raw_data[:,-1]

In [3]:
#balancing priors
num_one_target = int(np.sum(targets_all))
zero_target_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_target_counter+= 1
        if zero_target_counter > num_one_target:
            indices_to_remove.append(i)
unscaled_inputs_equal_prior = np.delete(unscaled_inputs,indices_to_remove,axis=0)
targets_equal_prior = np.delete(targets_all,indices_to_remove,axis=0)

In [4]:
#standardize inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_prior)

In [5]:
#shuffling
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_target = targets_equal_prior[shuffled_indices]

In [10]:
#splitting
sample_count = shuffled_inputs.shape[0]
train_samples_count = int(0.7*sample_count)
validation_samples_count = int(0.2*sample_count)
test_samples_count = sample_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_target = shuffled_target[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_target = shuffled_target[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_target = shuffled_target[train_samples_count+validation_samples_count:]

In [7]:
#saving as .npz
np.savez('Audiobooks_train_data',inputs=train_inputs,targets = train_target)
np.savez('Audiobooks_validation_data',inputs=validation_inputs,targets = validation_target)
np.savez('Audiobooks_test_data',inputs=test_inputs,targets = test_target)